<section id="sec1"> </section>
<h1> Imports </h1>

<a href="#back" style="text-decoration: none; color: #333;">Back to table of contents</a>

In [ ]:
# Basic imports
import numpy as np
import pandas as pd
import matplotlib as plt
from tqdm import tqdm

##################### sklearn imports 
from scipy.optimize import minimize
from sklearn.metrics import make_scorer
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedStratifiedKFold
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingRegressor

##################### Models
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

##################### optuna library import
import optuna
import shap
random_state = 42
n_splits = 5

c:\Users\Nadav\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
!pip install optuna
!pip install shap
import shap


[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
######## Download the data 
train=pd.read_csv("../data/processed/train_processed.csv")
test=pd.read_csv("../data/processed/test_processed.csv")
sample = pd.read_csv('../data/raw/sample_submission.csv')


train = train.dropna(subset='sii').reset_index().drop('index',axis=1)
train = train.select_dtypes(include='number')
train

,Basic_Demos-Age,Basic_Demos-Sex,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Systolic_BP,FGC-FGC_CU,FGC-FGC_GSND,FGC-FGC_GSD,FGC-FGC_PU,FGC-FGC_SRL_Zone,FGC-FGC_TL,BIA-BIA_FFMI,BIA-BIA_Frame_num,SDS-SDS_Total_Raw,PreInt_EduHx-computerinternet_hoursday,sii
0,5.0,0,16.877316,46.0,50.8,NaN,NaN,0.0,NaN,NaN,0.0,0.0,6.0,13.8177,1.0,NaN,3.0,2.0
1,9.0,0,14.035590,48.0,46.0,22.0,122.0,3.0,NaN,NaN,5.0,1.0,3.0,12.8254,1.0,46.0,0.0,0.0
2,10.0,1,16.648696,56.5,75.6,NaN,117.0,20.0,10.2,14.7,7.0,1.0,5.0,NaN,NaN,38.0,2.0,0.0
3,9.0,0,18.292347,56.0,81.6,NaN,117.0,18.0,NaN,NaN,5.0,0.0,7.0,14.0740,2.0,31.0,0.0,1.0
4,13.0,1,22.279952,59.5,112.2,NaN,102.0,12.0,16.5,17.9,6.0,1.0,8.0,16.6877,2.0,40.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2731,8.0,0,17.139810,52.5,67.2,25.0,112.0,0.0,NaN,NaN,0.0,1.0,12.0,13.4004,1.0,41.0,2.0,0.0
2732,7.0,1,13.927006,48.5,46.6,23.0,105.0,0.0,NaN,NaN,0.0,0.0,4.5,13.2315,1.0,48.0,0.0,1.0
2733,13.0,0,16.362460,59.5,82.4,NaN,104.0,16.0,18.0,19.9,10.0,1.0,12.0,14.0629,1.0,35.0,1.0,1.0
2734,11.0,0,21.441500,60.0,109.8,NaN,116.0,15.0,18.5,15.8,0.0,1.0,14.0,14.8043,2.0,56.0,0.0,1.0


In [ ]:
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

In [26]:
# Function to plot learning curves
def plot_learning_curve(model, X, y, cv):
    train_sizes, train_scores, valid_scores = learning_curve(model, X, y, cv=cv,
                                                             scoring=scorer,
                                                             train_sizes=np.linspace(0.1, 1.0, 5),
                                                             n_jobs=-1)

    train_scores_mean = np.mean(train_scores, axis=1)  # Convert from 
    valid_scores_mean = np.mean(valid_scores, axis=1)

    plt.figure()
    plt.plot(train_sizes, train_scores_mean, label="Training QWK")
    plt.plot(train_sizes, valid_scores_mean, label="Cross-Validation QWK")
    plt.title(f"Learning Curve ({model.__class__.__name__})")
    plt.xlabel("Training Set Size")
    plt.ylabel("QWK")
    plt.legend(loc="best")
    plt.grid(True)
    plt.show()

# Optuna optimization functions for each model
def objective_catboost(trial, X_train, y_train):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10),
        'random_strength': trial.suggest_uniform('random_strength', 1, 20),
        'bagging_temperature': trial.suggest_uniform('bagging_temperature', 0, 1),
        'border_count': trial.suggest_int('border_count', 1, 255),
    }

    model = CatBoostRegressor(**params, verbose=0, early_stopping_rounds=50)
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    score = cross_val_score(model, X_train, y_train, scoring=scorer, cv=cv, n_jobs=-1)

    return np.mean(score)  # Optuna tries to minimize, so we negate RMSE


def objective_xgboost(trial, X_train, y_train):
    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
    }

    model = XGBRegressor(**params, early_stopping_rounds=50)
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    score = cross_val_score(model, X_train, y_train, scoring=scorer, cv=cv, n_jobs=-1)

    return np.mean(score)

def objective_lightgbm(trial, X_train, y_train):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
    }

    model = LGBMRegressor(**params, verbose=-1, early_stopping_rounds=50)
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    score = cross_val_score(model, X_train, y_train, scoring=scorer, cv=cv, n_jobs=-1)

    return np.mean(score)

def objective_randomforest(trial, X_train, y_train):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2']),
    }

    model = RandomForestRegressor(**params)
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    score = cross_val_score(model, X_train, y_train, scoring=scorer, cv=cv, n_jobs=-1)

    return np.mean(score)

def objective_gradientboosting(trial, X_train, y_train):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
    }

    model = GradientBoostingRegressor(**params)
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    score = cross_val_score(model, X_train, y_train, scoring=scorer, cv=cv, n_jobs=-1)

    return np.mean(score)

In [27]:
y = train["sii"]
X = train.drop("sii",axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scorer = make_scorer(cohen_kappa_score, weights='quadratic')

def optimize_model(objective_func, n_trials=50):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective_func(trial, X_train, y_train), n_trials=n_trials)
    
    print(f"Best trial:")
    trial = study.best_trial
    print(f"  Value: {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")
    
    return study.best_params

# Optimize each model
models = {
    'CatBoost': objective_catboost,
    'XGBoost': objective_xgboost,
    'LightGBM': objective_lightgbm,
    'RandomForest': objective_randomforest,
    'GradientBoosting': objective_gradientboosting
}

best_params = {}
for model_name, objective_func in models.items():
    print(f"\nOptimizing {model_name}...")
    best_params[model_name] = optimize_model(objective_func)

[I 2024-12-09 20:34:07,037] A new study created in memory with name: no-name-2172b352-db50-45a2-8155-f22f1155731c



Optimizing CatBoost...


[W 2024-12-09 20:34:31,072] Trial 0 failed with parameters: {'iterations': 117, 'depth': 5, 'learning_rate': 0.024214825855439156, 'l2_leaf_reg': 0.10413526923476847, 'random_strength': 7.046615826508772, 'bagging_temperature': 0.501125163279096, 'border_count': 63} because of the following error: The value nan is not acceptable.
[W 2024-12-09 20:34:31,075] Trial 0 failed with value nan.
[W 2024-12-09 20:34:52,247] Trial 1 failed with parameters: {'iterations': 697, 'depth': 5, 'learning_rate': 0.01832655566307815, 'l2_leaf_reg': 7.657084794535111, 'random_strength': 18.763234072057553, 'bagging_temperature': 0.7340557250167444, 'border_count': 245} because of the following error: The value nan is not acceptable.
[W 2024-12-09 20:34:52,250] Trial 1 failed with value nan.
[W 2024-12-09 20:35:28,174] Trial 2 failed with parameters: {'iterations': 618, 'depth': 8, 'learning_rate': 0.016259796485650627, 'l2_leaf_reg': 0.09882183113040095, 'random_strength': 6.403393013808994, 'bagging_temp

KeyboardInterrupt: 

In [ ]:
def tune_model_with_learning_curve(model_name, X_train, y_train):
    if model_name == 'CatBoost':
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective_catboost(trial, X_train, y_train), n_trials=50)

        best_params = study.best_params
        print(f"Best parameters for {model_name}: {best_params}")

        # Train model with best parameters and plot learning curve
        model = CatBoostRegressor(**best_params, verbose=0, early_stopping_rounds=50)
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        plot_learning_curve(model, X_train, y_train, cv)

        return best_params

    elif model_name == 'XGBoost':
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective_xgboost(trial, X_train, y_train), n_trials=50)

        best_params = study.best_params
        print(f"Best parameters for {model_name}: {best_params}")

        model = XGBRegressor(**best_params)
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        plot_learning_curve(model, X_train, y_train, cv)

        return best_params

    elif model_name == 'LightGBM':
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective_lightgbm(trial, X_train, y_train), n_trials=50)

        best_params = study.best_params
        print(f"Best parameters for {model_name}: {best_params}")

        model = LGBMRegressor(**best_params)
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        plot_learning_curve(model, X_train, y_train, cv)

        return best_params

    elif model_name == 'RandomForest':
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective_randomforest(trial, X_train, y_train), n_trials=50)

        best_params = study.best_params
        print(f"Best parameters for {model_name}: {best_params}")

        model = RandomForestRegressor(**best_params)
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        plot_learning_curve(model, X_train, y_train, cv)

        return best_params

    elif model_name == 'GradientBoosting':
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective_gbr(trial, X_train, y_train), n_trials=50)

        best_params = study.best_params
        print(f"Best parameters for {model_name}: {best_params}")

        model = GradientBoostingRegressor(**best_params)
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        plot_learning_curve(model, X_train, y_train, cv)

        return best_params

    elif model_name == 'LinearRegression':
        print("No hyperparameters to tune for Linear Regression.")
        model = LinearRegression()
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        plot_learning_curve(model, X_train, y_train, cv)

        return "No hyperparameters tuned for Linear Regression."